# Project 1: Data analysis

## 1a) Bayesian approch

In order to predict the effect of genes on symptoms, we will use a simple Bayesian model. We consider these hypothesis:

$$\mu_0: P(symptoms|genes) = P(symptoms)$$
$$\mu_1: P(symptoms|genes) \neq P(symptoms)$$

In order words, does the genes have any effect on the symptoms or not? If it does, which genomes are the most relevant for the preventing the symptoms? 


In our case our Y is the symptoms and our X is the genes. Y is an 8 bit vector for each observation,with 1 or zero for each bit. X is an 128 with the same charectaristic as Y.


The probability distribution of $ Y $ depends on an unknown parameter which also is stocastic. In this case the unknown parameter is $\theta$ or $ p $. 

The formulation under is an alternative formulation of the the previous hypotesis.Does the parameter that the probability dist of Y depend on, depend on what the value of X is or not. $ \mu_0 $ says that it does not.$\mu_1 $ says it does.
 
$$\mu_0: \theta^0 \sim Beta(\alpha^0 , \beta^0) ,  \ \ \  Y/ \theta^0  $$
$$\mu_1: \theta^{1,x_t} \sim Beta(\alpha^{1,x_t} , \beta^{1,x_t}) , \ \ \ y_t/x_t \sim bernulli(\theta^{1,x_t}) $$
 

We will model the hypothesis in the following way. Given the data, we try to estimate

$$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_1}(D) \cdot \xi(\mu_1)} $$

We can eather set $P(\mu | D)$ to mean $P(\mu_1 | D)$ or  $P(\mu_0 | D)$ depending on which hypothesis we want to check the posterior belief of. 

where $P_{\mu} (D) = P(\mu |D)$, $D = (x_t, y_t,x_{t-1},y_{t-1})_{t=1} ^T $ for a spesific gene, $x_t$ is the value of the gene for observation t ,$x_{t-1}$ is a vector of all the previous  values of the previous observation up to t-1,  $y_t$ is the the value of symptom Y for observation t,$y_{t-1}$ is a vector of all the previous  values of the previous symtoms observation up to t-1 and $\mu$ is one of the hypothesis in the set {$\mu_1$,$\mu_0$}.  We assume that the different observations(rows) of the data are independent of each other. This gives us:
$$ P_{\mu}(D) = P(y_1,...,y_t \cup x_1,...,x_t)$$

$$ = \prod_{i=1}^{t} P(y_i | x_i,x_{i-1} , y_{i-1}) $$


With these assumptions, we can estimate the terms in the $P_{\mu} (D)$ by 
$$ P(y_t | x_t,x_{t-1} , y_{t-1}) = \int_{\theta} P_{\theta}(y_t|x_t) \cdot \xi(\theta|x_{t-1},y_{t-1})$$

We calculate the likehood of adding a new observation(row) using this formula.It works if we assume that the the next observation is independent from the previous observations.


This is the marginal likelihood(posterior predictive distribution) which is a compound distribution,which is used to calculate $P(y_t | x_{t},x_{t-1}, y_{t-1})$. The probabilty $P(y_t | x_{t})$ is stocatic in $\theta$,so we have to integrate over all values of $\theta$. Every time a new observation is added we calculate $P(y_t | x_{t},x_{t-1}, y_{t-1})$,and recalculate $$ = \prod_{i=1}^{t} P(y_i | x_i ,x_{i-1}, y_{i-1}) $$. 

So to calculate $$ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_i}(D) \cdot \xi(\mu_i)} $$,we are updating our belief,every time a new observation is added,sequentially. For every update $\xi(\mu)$ is $P(\mu | D)$ from the previous iteration,exept the first iteration where $\xi(\mu)$ cancels out because it's constant.

This integral simplifies to:

$$ P(y_t | x_{t},x_{t-1},y^{t-1}) = \int_{\theta} P_{\theta}(y_t) \cdot d \xi(\theta|y^{t-1}) = \frac{\alpha_{t,x_t}}{\alpha_{t,x_t} + \beta_{t,x_t}}$$


Looking at $\mu_1$ we are using the beta-binomial distribution.Its a binomial distribution where  the probability of success at each of n trials is not fixed but comes from a beta distribution. The bernulli distribtuion is a special case of the binomial distribution,where number of trials is equal to 1.

We are doing this because our y's are either 1 or 0.Then it is reasonable to assume that Y/X is bernulli distributed.

It is also conviniant because of its conjugate prior property.Now we can calculate  $P( D/\mu_1) =\prod_{i=1}^{t} P(y_i | x^i ,x^{i-1}, y^{i-1}) $ beacuse we know that the marginal likelihood (posterior predictive dist) is composed of a binomial dist times a beta dist $ P(y_t | x^t , y^{t-1}) = \int_{\theta} P_{\theta}(y_t|x_t) \cdot \xi(\theta|x^{t-1},y^{t-1})$,which is beta distributed .For each iteration we end up  getting that $P_{\mu} (D)$ is beta distributed.

The same integral calculation is done for $P(D | \mu_0)$ but now the marginal likelihood is independent of the data.

Instead of calculating $P(  D|\mu_1) =\prod_{i=1}^{t} P(y_i | x_i ,x_{i-1}, y_{i-1})$ ,we can first calculate $log(P(  D|\mu_1))$ and then do $e^{log(P(  D|\mu_1))}$,same for $P(D | \mu_0)$.

To summarize.We have a set of hypotesis in this case two.We want to find how accurate they are. So we have a prior beleif on the strenght of the hypothses which we express through a probability, and we want to update our belief on the strenght of the hypotesis(posterior belief). In other word what is the probability of the choosen hypotesis given the data.Ony have to compute one of the probabilities because $P(\mu_1 | D)=1-P(\mu_0 | D)$.

The calculation is done sequentially.For each iteration we get a new data observation.A new row of genes.We continously update the posterior belief,for all genes,and put the posterior belief of each gene into a list. The belief get updated through calculating the marginal likelihood every time a new row is added and a product is taken between this marginal likelihood and all the presious marginal likelihood calculations bought for $P(  D|\mu_1)$ and for$P(  D|\mu_0)$ this is then put into the bayes formula $ P(\mu | D) = \frac{P_{\mu} (D) \cdot \xi(\mu)}{\sum_{i=1} ^n P_{\mu_i}(D) \cdot \xi(\mu_i)} $.

We want to find this probability for for each of the genes.This procedure is done for each gene.For each gene if the posterior probabilty is greater then some trechhold we add this gene to the list of important genes that are relvant for explaining that spesific symptom.
 



The decision rule can be defined as

$$ \mu =   \left\{
\begin{array}{ll}
      \mu_1 & P(\mu_1|D) > s \\
      \mu_0 & else
\end{array} 
\right.  $$

where $s$ is a given threshold.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [140]:
cols = (['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death'] + 
        ['Age', 'Gender', 'Income'] + 
         [f'Gene_{i+1:03}' for i in range(128)] + 
         ['Asthma', 'Obesity', 'Smoking', 'Diabetes', 'Heart disease', 'Hypertension'] +
         ['Vacc_1', 'Vacc_2', 'Vacc_3'])

In [143]:
obs_data = pd.read_csv("observation_features.csv")
treat_data = pd.read_csv("treatment_features.csv") 
action_data = pd.read_csv("treatment_actions.csv")
outcome_data = pd.read_csv("treatment_outcomes.csv") 

obs_data.columns = cols
treat_data.columns = cols
outcome_data.columns = cols[:10]
action_data.columns = ['Treatment_1', 'Treatment_2']

In [4]:
symptoms = np.array(obs_data.iloc[:,0:10])
age = obs_data.iloc[:,10]
gender = obs_data.iloc[:,11]
income = obs_data.iloc[:,12]
genome = obs_data.iloc[:,13:141]
comorbidities = obs_data.iloc[:,141:147]
vaccination_status = np.array(obs_data.iloc[:,147:])

In [5]:
def select_features(X, Y, threshold):
    n_features = X.shape[1]
    n_data =  X.shape[0]
    alpha_b = np.ones([n_features, 2 ])
    beta_b = np.ones([n_features, 2])
    log_p = np.zeros(n_features)
    
    log_null = 0
    alpha = 1
    beta = 1  
    for t in range(n_data):
        p_null = alpha / (alpha + beta)
        log_null += np.log(p_null)*Y[t] + np.log(1-p_null)*(1 - Y[t])
        alpha += Y[t]
        beta += (1 - Y[t])
        for i in range(n_features):
            
                x_ti = int(X[t,i])
                p = alpha_b[i, x_ti] / (alpha_b[i, x_ti] + beta_b[i, x_ti])
                log_p[i] += np.log(p)*Y[t] + np.log(1-p)*(1 - Y[t])
                alpha_b[i, x_ti] += Y[t]
                beta_b[i, x_ti] += (1 - Y[t])
    log_max=np.mean(log_p)
    log_max2=np.mean(log_null)
    log_p=log_p-log_max
    log_null=log_null-log_max2
    p = np.exp(log_p) / (np.exp(log_p) + np.exp(log_null))
    #print((np.exp(log_p) + np.exp(log_null)))

    features = []
    for i in range(n_features):
        if p[i] > threshold:
            features.append(i)
    print(alpha)
    print(beta)
    return features

In [144]:
select_0 = select_features(np.array(genome),symptoms[:,1],0.8)

KeyboardInterrupt: 

In [159]:
select_0

NameError: name 'select_0' is not defined

## 1b) Estimating the efficacy of vaccines

We estimate the efficacy of the vaccines by
$$ 100 \cdot (1-IRR)$$
where IRR is the ratio of the vaccinated incidence rate and not vaccinated incidence incidenes ([link](https://www.nejm.org/doi/full/10.1056/NEJMoa2034577)). 

As above, we use a Bayesian Beta-Binomial model. Hence, our prior belief is assumed to have a Beta distribution and the Likelihood of observed data is Binomial distributed. This result in posterior is Beta-distributed (more details will follow afterwards).

We set a low beta value for the prior distribution and 

In [9]:
def count_status(vac_list, symptom_name):
    index = symptom_names.index(symptom_name)
    vacc_pos = 0
    for i in vac_list:
        if symptoms[i][1] == 1 and symptoms[i][index] == 1: 
            vacc_pos += 1
    return vacc_pos

def find_alpha(beta,p):
    return beta*p/(1-p)

def find_efficacy(vaccinated,not_vaccinated,symptom_name,count_status):
    vacc_pos = count_status(vaccinated,symptom_name)
    not_pos = count_status(not_vaccinated,symptom_name)

    v = vacc_pos/len(vaccinated)
    n_v = not_pos/len(not_vaccinated)
    IRR = v/n_v
    
    efficacy = 100*(1- IRR)
    
    N = len(obs_data)
    beta = 1
    p = prior_probs[symptom_name]
    alpha = find_alpha(beta,p)
    
    samples_vaccine = stats.beta.rvs(alpha + vacc_pos, beta + len(vaccinated) - vacc_pos, size=N)
    samples_no_vaccine = stats.beta.rvs(alpha + not_pos, beta + len(not_vaccinated) - not_pos, size=N)
    
    samples_ve = 100 * (1 - samples_vaccine/samples_no_vaccine)
    lower = np.percentile(samples_ve, 2.5)
    upper = np.percentile(samples_ve, 97.5)
    print(f'{symptom_name:15s}: {efficacy:3.3f} - ({lower:3.3f}, {upper:3.3f})')    

In [10]:
#samples with vaccinations or not:
not_vaccinated = [i for i in range(len(vaccination_status)) if all(v==0 for v in vaccination_status[i])]
vaccinated = [i for i in range(len(vaccination_status)) if any(v !=0 for v in vaccination_status[i])]
symptom_names = ['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death']
prior_probs= {key:np.sum(obs_data.iloc[:,i]) / len(obs_data) for i, key in enumerate(symptom_names)}

print("Efficacy of the vaccinated:")
for s in symptom_names:
    find_efficacy(vaccinated,not_vaccinated,s,count_status)
    


Efficacy of the vaccinated:
Covid-Recovered: 28.965 - (20.087, 36.824)
Covid-Positive : 21.464 - (19.607, 23.264)
No-Taste/Smell : 47.521 - (41.623, 52.882)
Fever          : 51.575 - (44.983, 57.439)
Headache       : 45.473 - (28.938, 58.320)
Pneumonia      : 57.810 - (51.555, 63.339)
Stomach        : 50.542 - (26.010, 67.514)
Myocarditis    : 44.531 - (28.525, 57.045)
Blood-Clots    : 57.190 - (50.893, 62.733)
Death          : 93.394 - (90.535, 95.660)


In [62]:
def find_efficacy(group_pos: pd.DataFrame, group_neg: pd.DataFrame, symptom_index: int, prior_probs):
    group_pos_count = np.sum(group_pos.iloc[:,symptom_index] * group_pos.iloc[:,1])
    group_neg_count = np.sum(group_neg.iloc[:,symptom_index] * group_neg.iloc[:,1])

    v = group_pos_count/len(group_pos)
    n_v = group_neg_count/len(group_neg)
    
    if n_v == 0:
        print('Division by zero')
        return
    
    IRR = v/n_v
    
    print(v, n_v)
    efficacy = 100*(1- IRR)
    
    N = len(group_pos) + len(group_neg)
    beta = 1
    p = prior_probs[symptom_index]
    alpha = find_alpha(beta,p)
    
    symptom_name = symptom_names[symptom_index]
    
    samples_group_pos = stats.beta.rvs(alpha + group_pos_count, beta + len(group_pos) - group_pos_count, size=N)
    samples_group_neg = stats.beta.rvs(alpha + group_neg_count, beta + len(group_neg) - group_neg_count, size=N)
    
    samples_ve = 100 * (1 - samples_group_pos/samples_group_neg)
    lower = np.percentile(samples_ve, 2.5)
    upper = np.percentile(samples_ve, 97.5)
    print(f'{symptom_name:15s}: {efficacy:3.3f} - ({lower:3.3f}, {upper:3.3f})')    

In [12]:
obs_data.iloc[:,9] * obs_data.iloc[:,1]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99994    0.0
99995    0.0
99996    0.0
99997    0.0
99998    0.0
Length: 99999, dtype: float64

In [13]:
cols = symptom_names + (['Age', 'Gender', 'Income']+ 
                 [f'Gene_{i+1:03}' for i in range(128)] + 
                 ['Asthma', 'Obesity', 'Smoking', 'Diabetes', 'Heart disease', 'Hypertension'] +
                 ['Vacc_1', 'Vacc_2', 'Vacc_3'])

obs_data.columns = cols
obs_data.Fever

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99994    0.0
99995    0.0
99996    0.0
99997    1.0
99998    0.0
Name: Fever, Length: 99999, dtype: float64

In [14]:
vacced

NameError: name 'vacced' is not defined

In [15]:
#samples with vaccinations or not:
vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 1]
vacced_neg = vacced[vacced.iloc[:,1]==0]
vacced_pos = vacced[vacced.iloc[:,1]==1]

un_vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 0]
un_vacced_neg = un_vacced[un_vacced.iloc[:,1]==0]
un_vacced_pos = un_vacced[un_vacced.iloc[:,1]==1]

symptom_names = ['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death']
prior_probs= [np.sum(obs_data.iloc[:,i]) / len(obs_data) for i, key in enumerate(symptom_names)]

print("Efficacy of the vaccinated:")
for i, s in enumerate(symptom_names):
    find_efficacy(vacced,un_vacced,i,prior_probs)
    

Efficacy of the vaccinated:
Covid-Recovered: 28.965 - (20.109, 36.845)
Covid-Positive : 21.464 - (19.615, 23.270)
No-Taste/Smell : 47.521 - (41.657, 52.870)
Fever          : 51.575 - (44.933, 57.479)
Headache       : 45.473 - (28.945, 58.290)
Pneumonia      : 57.810 - (51.617, 63.310)
Stomach        : 50.542 - (25.714, 67.416)
Myocarditis    : 44.531 - (28.541, 57.070)
Blood-Clots    : 57.190 - (50.870, 62.727)
Death          : 93.394 - (90.511, 95.668)


In [412]:
vacc_type1 = [i for i in range(len(vaccination_status)) if vaccination_status[i][0] == 1]
vacc_type2 = [i for i in range(len(vaccination_status)) if vaccination_status[i][1] == 1]
vacc_type3 = [i for i in range(len(vaccination_status)) if vaccination_status[i][2] == 1]
vaccination_types = [vacc_type1,vacc_type2,vacc_type3]
vaccination_names = ['type 1', 'type 2', 'type 3']

for name in vaccination_names:
    print(name)
    index = vaccination_names.index(name)
    for s in symptom_names:
        find_efficacy(vaccination_types[index],not_vaccinated,s,count_status)
    print("")

type 1
Covid-Recovered: 19.633 - (6.035, 31.587)
Covid-Positive : 11.296 - (8.499, 14.017)
No-Taste/Smell : 34.419 - (24.587, 43.290)
Fever          : 43.590 - (32.867, 53.011)
Headache       : 30.767 - (2.908, 52.311)
Pneumonia      : 54.129 - (44.167, 62.734)
Stomach        : 28.704 - (-16.856, 59.329)
Myocarditis    : 36.093 - (10.370, 55.765)
Blood-Clots    : 48.746 - (38.209, 57.960)
Death          : 95.704 - (91.567, 98.429)

type 2
Covid-Recovered: 28.837 - (16.347, 39.887)
Covid-Positive : 20.584 - (17.991, 23.129)
No-Taste/Smell : 43.109 - (34.130, 51.281)
Fever          : 45.703 - (35.194, 54.930)
Headache       : 39.907 - (14.319, 59.395)
Pneumonia      : 48.345 - (37.721, 57.528)
Stomach        : 70.324 - (42.878, 87.556)
Myocarditis    : 35.284 - (9.578, 55.129)
Blood-Clots    : 47.762 - (37.188, 57.031)
Death          : 92.922 - (87.681, 96.638)

type 3
Covid-Recovered: 38.211 - (26.739, 48.194)
Covid-Positive : 32.271 - (29.892, 34.594)
No-Taste/Smell : 64.686 - (57.866,

## 1c) Estimating the probability of vaccination side-effects.

Using the Bayes theorem, we will estimate the probability of vaccination side-effects. In other words, the probability of getting symptoms given that a person is vaccinated and has tested negative.
$$p_1 = P(symptom | vaccine,covid') = \frac{P(symptom \cap vaccine,covid')}{P(vaccine,covid')} $$
$$p_2 = P(symptom | vaccine',covid') = \frac{P(symptom \cap vaccine',covid')}{P(vaccine',covid')} $$

We want to test the following hypothesis:
$$ h_0: p_1 - p_2 \leq 0 $$
$$h_a: p_1 - p_2 > 0 $$

We make a confidence intervall for each symptom, and reject the null hypothesis if confidence intervall does not include zero. 

In [413]:
vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 1]
vacced_neg = vacced[vacced.iloc[:,1]==0]

un_vacced = obs_data[np.sum(obs_data.iloc[:,-3:], axis=1) == 0]
un_vacced_neg = un_vacced[un_vacced.iloc[:,1]==0]

for i in range(2, 10):
    p_1 = vacced_neg.sum()[i] / len(obs_data) / (len(vacced_neg) / len(obs_data))
    p_2 = un_vacced_neg.sum()[i] / len(obs_data) / (len(un_vacced_neg) / len(obs_data))

    print(f'{symptom_names[i]:15s}:{p_1:4.5f}',end=' ')
    print(f'{p_2:4.5f}',end=' ')
    print(f' diff:{p_1-p_2:4.5f}')

No-Taste/Smell :0.00081 0.00057  diff:0.00024
Fever          :0.09903 0.00508  diff:0.09395
Headache       :0.05579 0.01053  diff:0.04526
Pneumonia      :0.00133 0.00144  diff:-0.00011
Stomach        :0.00231 0.00257  diff:-0.00026
Myocarditis    :0.00214 0.00047  diff:0.00168
Blood-Clots    :0.00243 0.00110  diff:0.00133
Death          :0.00035 0.00000  diff:0.00035


**A large-sample 95% confidence interval for $p_1 - p_2$**:

In [414]:
vac_probs = [(vacced_neg.sum()[i] / len(obs_data)) / (len(vacced_neg) / len(obs_data)) for i in range(2,10)]
unvac_probs = [(un_vacced_neg.sum()[i] / len(obs_data)) / (len(un_vacced_neg) / len(obs_data)) for i in range(2,10)]
list(zip(vac_probs, unvac_probs))
np.array(vac_probs) - np.array(unvac_probs)

i = 2
for p1, p2 in zip(vac_probs, unvac_probs):
    lower = (p1-p2 - 1.64 * np.sqrt((p1*(1-p1) / len(vacced_neg)) + (p2 * (1-p2) / len(un_vacced_neg))))
    higher = (p1-p2 + 1.64 * np.sqrt((p1*(1-p1) / len(vacced_neg)) + (p2 * (1-p2) / len(un_vacced_neg))))
    print(f"{symptom_names[i]:15s}: ({lower:8.5f},{higher:6.5f})")
    i += 1

No-Taste/Smell : (-0.00007,0.00055)
Fever          : ( 0.09161,0.09628)
Headache       : ( 0.04329,0.04723)
Pneumonia      : (-0.00056,0.00035)
Stomach        : (-0.00086,0.00034)
Myocarditis    : ( 0.00127,0.00208)
Blood-Clots    : ( 0.00085,0.00182)
Death          : ( 0.00021,0.00049)


In [135]:
treat_data

,Covid-Recovered,Covid-Positive,No-Taste/Smell,Fever,Headache,Pneumonia,Stomach,Myocarditis,Blood-Clots,Death,...,Gene_128,Asthma,Obesity,Smoking,Diabetes,Heart disease,Hypertension,Vacc_1,Vacc_2,Vacc_3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Index(['0.0', '1.0'], dtype='object')

In [146]:
new_treat_data = treat_data.copy()[((np.sum(treat_data.iloc[:,2:10],axis=1) > 0.0) 
              & np.sum(outcome_data.iloc[:,2:10],axis=1) > 0.0)]
new_treat_data

,Covid-Recovered,Covid-Positive,No-Taste/Smell,Fever,Headache,Pneumonia,Stomach,Myocarditis,Blood-Clots,Death,...,Gene_128,Asthma,Obesity,Smoking,Diabetes,Heart disease,Hypertension,Vacc_1,Vacc_2,Vacc_3
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
967,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
970,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
989,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
group_first = treat_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
group_second = treat_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
group_both = treat_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
group_none = treat_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]

The relation we are modelling is whether any of the individuals who had symptoms do not have those symptoms after treatment or got symptoms from the treatment. 

We are not interested in the outcomes for individuals who did not have any symptoms 

In [147]:
new_outcome_data = outcome_data.copy()[((np.sum(treat_data.iloc[:,2:10],axis=1) > 0.0) 
              & np.sum(outcome_data.iloc[:,2:10],axis=1) > 0.0)]
new_outcome_data

,Covid-Recovered,Covid-Positive,No-Taste/Smell,Fever,Headache,Pneumonia,Stomach,Myocarditis,Blood-Clots,Death
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
15,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
920,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
967,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
970,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
989,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
outcome_first = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
outcome_second = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
outcome_both = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
outcome_none = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]

C:\Users\Tellef\AppData\Local\Temp/ipykernel_1620/4031450266.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  outcome_first = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 0))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_1620/4031450266.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  outcome_second = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 1))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_1620/4031450266.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  outcome_both = new_outcome_data[((action_data.iloc[:,0] == 1) & (action_data.iloc[:,1] == 1))]
C:\Users\Tellef\AppData\Local\Temp/ipykernel_1620/4031450266.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  outcome_none = new_outcome_data[((action_data.iloc[:,0] == 0) & (action_data.iloc[:,1] == 0))]


In [158]:
outcome_both

,Covid-Recovered,Covid-Positive,No-Taste/Smell,Fever,Headache,Pneumonia,Stomach,Myocarditis,Blood-Clots,Death
66,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [56]:
any(v != 0 for l in ((group_first.iloc[:,2:10]) - outcome_first.iloc[:,2:10]).values for v in l)

False

In [63]:
find_efficacy(group_first.iloc[:,2:9] - outcome_first.iloc[:,2:9],
            group_none.iloc[:,2:9] - outcome_none.iloc[:,2:9],
            3,
            [0.3]*10)

0.0 0.0
Fever          : nan - (-149361.811, 99.884)


C:\Users\Tellef\AppData\Local\Temp/ipykernel_1620/3220831272.py:7: RuntimeWarning: invalid value encountered in double_scalars
  IRR = v/n_v


In [139]:
print(symptom_names)

['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death']


In [70]:
(treat_data.iloc[:,2:10]).sample(10000, replace=True, random_state=42).sum()

0.0.2    145.0
0.0.3    221.0
0.0.4     53.0
0.0.5    109.0
0.0.6      6.0
0.0.7     35.0
0.0.8     71.0
0.0.9     42.0
dtype: float64

In [102]:
probs = [0.0, 0.6, 0.3, 0.1, 0.0, 0.1, 0.5, 0.0]
new_outcomes = outcome_data.iloc[:, 2:10]
np.random.choice(2, p=(0.2,0.8))

for index, key in enumerate(new_outcomes):
    for i, val in enumerate(new_outcomes[key]):
        if val == 1:
            new_outcomes[key][i] = np.random.choice(2, p=(probs[index], 1- probs[index]))
            
    

C:\Users\Tellef\AppData\Local\Temp/ipykernel_1620/1222785870.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_outcomes[key][i] = np.random.choice(2, p=(probs[index], 1- probs[index]))


In [76]:
np.random.choice?

In [104]:
new_outcomes.sum()

0.0.2    14.0
0.0.3     7.0
0.0.4     5.0
0.0.5     8.0
0.0.6     1.0
0.0.7     4.0
0.0.8     3.0
0.0.9     4.0
dtype: float64

In [107]:
treat_data.sum()[2:10]

0.0.2    14.0
0.0.3    22.0
0.0.4     6.0
0.0.5    11.0
0.0.6     1.0
0.0.7     5.0
0.0.8     7.0
0.0.9     4.0
dtype: float64